# Content based movie recommender

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import euclidean_distances
import warnings
warnings.filterwarnings("ignore")
data = pd.read_csv('movie_dataset.csv')

In [2]:
data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,Unnamed: 1255,Unnamed: 1256,Unnamed: 1257,Unnamed: 1258,Unnamed: 1259,Unnamed: 1260,Unnamed: 1261,Unnamed: 1262,Unnamed: 1263,Unnamed: 1264
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.31295,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Dropping unnecessary features

In [3]:
features = ['keywords','cast','genres','director','title','index']

In [4]:
data.drop(data.columns.difference(features), 1, inplace=True)

The function dataframe.columns.difference() gives you complement of the values that you provide as argument. It can be used to create a new dataframe from an existing dataframe with exclusion of some columns.

In [5]:
data

,index,genres,keywords,title,cast,director
0,0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Avatar,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,Pirates of the Caribbean: At World's End,Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,2,Action Adventure Crime,spy based on novel secret agent sequel mi6,Spectre,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Dark Knight Rises,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,John Carter,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton
...,...,...,...,...,...,...
4804,4798,Action Crime Thriller,united states\u2013mexico barrier legs arms pa...,El Mariachi,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,Robert Rodriguez
4805,4799,Comedy Romance,NaN,Newlyweds,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,Edward Burns
4806,4800,Comedy Drama Romance TV Movie,date love at first sight narration investigati...,"Signed, Sealed, Delivered",Eric Mabius Kristin Booth Crystal Lowe Geoff G...,Scott Smith
4807,4801,NaN,NaN,Shanghai Calling,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,Daniel Hsia


Dealing with null data

In [6]:
data.isnull().any()

index       False
genres       True
keywords     True
title        True
cast         True
director     True
dtype: bool

In [7]:
for feature in features:
    data[feature] = data[feature].fillna('')

### Creating a new feature that combines all the necessary features

In [8]:
data['combined_features'] = data['genres'] + ' ' + data['keywords'] + ' ' + data['cast'] + ' ' + data['director']

In [9]:
data

,index,genres,keywords,title,cast,director,combined_features
0,0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Avatar,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron,Action Adventure Fantasy Science Fiction cultu...
1,1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,Pirates of the Caribbean: At World's End,Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski,Adventure Fantasy Action ocean drug abuse exot...
2,2,Action Adventure Crime,spy based on novel secret agent sequel mi6,Spectre,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes,Action Adventure Crime spy based on novel secr...
3,3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Dark Knight Rises,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan,Action Crime Drama Thriller dc comics crime fi...
4,4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,John Carter,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton,Action Adventure Science Fiction based on nove...
...,...,...,...,...,...,...,...
4804,4798,Action Crime Thriller,united states\u2013mexico barrier legs arms pa...,El Mariachi,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,Robert Rodriguez,Action Crime Thriller united states\u2013mexic...
4805,4799,Comedy Romance,,Newlyweds,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,Edward Burns,Comedy Romance Edward Burns Kerry Bish\u00e9 ...
4806,4800,Comedy Drama Romance TV Movie,date love at first sight narration investigati...,"Signed, Sealed, Delivered",Eric Mabius Kristin Booth Crystal Lowe Geoff G...,Scott Smith,Comedy Drama Romance TV Movie date love at fir...
4807,4801,,,Shanghai Calling,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,Daniel Hsia,Daniel Henney Eliza Coupe Bill Paxton Alan R...


### Using CountVectorizer() to get the bag-of-words for each movie

In [10]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(data['combined_features'])

Cosine similarity is used in information retrieval and text mining. It calculates the similarity between two vectors. If you have two documents and want to find the similarity between them you have to find the cosine angle between the two vectors to check similariy.

In [11]:
cosine_sim = cosine_similarity(count_matrix)

In [12]:
cosine_sim

array([[1.        , 0.10540926, 0.12038585, ..., 0.        , 0.        ,
        0.        ],
       [0.10540926, 1.        , 0.0761387 , ..., 0.03651484, 0.        ,
        0.        ],
       [0.12038585, 0.0761387 , 1.        , ..., 0.        , 0.11145564,
        0.        ],
       ...,
       [0.        , 0.03651484, 0.        , ..., 1.        , 0.        ,
        0.04264014],
       [0.        , 0.        , 0.11145564, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.04264014, 0.        ,
        1.        ]])

In [13]:
cosine_sim.shape

(4809, 4809)

## Recommending 10 movies based on the cosine similarity

In [14]:
def movie_recommend(title):
    index = data[data['title'] == title].index[0]
    sim_score = cosine_sim[index,:]
    a = list(enumerate(sim_score))
    sorted_similar_movies = sorted(a,key=lambda x:x[1],reverse = True)[1:]
    sim_index = []
    for i in range(10):
        idx = sorted_similar_movies[i][0]
        sim_index.append(idx)
    movie_names = []
    for i in sim_index:
        name = data.iloc[i]['title']
        movie_names.append(name)
    return movie_names

In [15]:
movie_recommend("Jurassic Park")

['The Lost World: Jurassic Park',
 'Jurassic Park III',
 'Jurassic World',
 'Close Encounters of the Third Kind',
 'A.I. Artificial Intelligence',
 'Independence Day: Resurgence',
 'Independence Day',
 'Event Horizon',
 'E.T. the Extra-Terrestrial',
 'War of the Worlds']